## 

### 基于RNN的文本分类

In [21]:
import numpy as np
import os
import time
import datetime

import tensorflow as tf
from tensorflow.contrib import rnn

from sklearn.utils import shuffle
from sklearn.cross_validation import train_test_split

#### 1.模型参数

In [22]:
# Parameters
# ==================================================

# Data loading params
tf.flags.DEFINE_float("dev_sample_percentage", .1, "Percentage of the training data to use for validation")
tf.flags.DEFINE_string("file_path", "thu_data_3class_3k", "Data source.")
tf.flags.DEFINE_integer("num_classes", 3, "number classes of datasets.")

# Model Hyperparameters
tf.flags.DEFINE_integer("embedding_dim", 200, "Dimensionality of character embedding (default: 128)")
tf.flags.DEFINE_float("dropout_keep_prob", 0.5, "Dropout keep probability (default: 0.5)")
tf.flags.DEFINE_integer("hidden_unit", 200, "number of RNN hidden cell")
tf.flags.DEFINE_string("cell", 'lstm', "Which RNN cell will be used (dedault: lstm)")
tf.flags.DEFINE_float("num_layer", 1, "RNN layers (default: 1)")
tf.flags.DEFINE_float("l2_reg_lambda", 0.0, "L2 regularization lambda (default: 0.0)")

# Training parameters
tf.flags.DEFINE_integer("batch_size", 32, "Batch Size (default: 64)")
tf.flags.DEFINE_integer("num_epochs", 100, "Number of training epochs (default: 200)")
tf.flags.DEFINE_integer("evaluate_every", 100, "Evaluate model on dev set after this many steps (default: 100)")
tf.flags.DEFINE_integer("checkpoint_every", 100, "Save model after this many steps (default: 100)")
tf.flags.DEFINE_integer("num_checkpoints", 5, "Number of checkpoints to store (default: 5)")
# Misc Parameters
tf.flags.DEFINE_boolean("allow_soft_placement", True, "Allow device soft device placement")
tf.flags.DEFINE_boolean("log_device_placement", False, "Log placement of ops on devices")

FLAGS = tf.flags.FLAGS
FLAGS._parse_flags()
print('\nParameters')
for attr, value in sorted(FLAGS.__flags.items()):
    print("{}={}".format(attr.upper(), value))
print("")


Parameters
ALLOW_SOFT_PLACEMENT=True
BATCH_SIZE=32
CELL=lstm
CHECKPOINT_EVERY=100
DEV_SAMPLE_PERCENTAGE=0.1
DROPOUT_KEEP_PROB=0.5
EMBEDDING_DIM=200
EVALUATE_EVERY=100
FILE_PATH=thu_data_3class_3k
HIDDEN_UNITS=200
L2_REG_LAMBDA=0.0
LOG_DEVICE_PLACEMENT=False
NUM_CHECKPOINTS=5
NUM_CLASSES=3
NUM_EPOCHS=100
NUM_RNN_LAYER=1



#### 2. 加载数据

In [23]:
import sys
# 这句仅仅是在pycharm中指定目录可以方便导包
sys.path.append(r'/Users/macan/Documents/pro/workspace/Python/MQNLP/text_classification') 

import Ipynb_impoter
import data_helper

In [3]:
input_x, input_y, word_index = data_helper.load_data(FLAGS.file_path, maxlen=400)
x_train, x_dev, y_train, y_dev = train_test_split(input_x, input_y,
                                                  test_size=FLAGS.dev_sample_percentage)


IndentationError: unexpected indent (<ipython-input-3-154b1197de0a>, line 3)

#### 4. 模型搭建

In [1]:
class RNNs(object):

    def __init__(self, embedding_mat, embedding_dims, vocab_size,
                 hidden_unit, sequence_length, num_classes, cell='lstm',
                 num_layer=1, l2_reg_lambda=0.0):
        """

        :param seq_length:
        :param embedding_dims:
        :param hidden_units:
        :param batch_size:
        :param epochs:
        :param cell:
        :param num_layer:
        :param l2_reg_lambda:
        """
        self.seq_length = sequence_length
        self.embedding_mat = embedding_mat
        self.vocab_size = vocab_size
        self.hidden_unit = hidden_unit
        self.embedding_dims = embedding_dims
        self.num_classes = num_classes
        self.cell = cell.lower()
        self.num_layer = num_layer 
        self.l2_reg_lambda = l2_reg_lambda

        # [样本个数，每个样本的词个数]
        self.input_x = tf.placeholder(tf.int32, [None, self.seq_length], name='input_x')
        # [样本个数， 类别个数]
        self.input_y = tf.placeholder(tf.float32, [None, self.num_classes], name='input_y')
        # dropout probability
        self.dropout_keep_prob = tf.placeholder(tf.float32, name='dropout_keep_prob')

        # l2 正则 损失
        self.l2_loss = tf.constant(0.0)

        self.network()

    def witch_cell(self):
        if self.cell == 'lstm':
            cell_tmp = rnn.BasicLSTMCell(self.hidden_unit)
        elif self.cell == 'gru':
            cell_tmp = rnn.GRUCell(self.hidden_unit)
        # 是否需要进行dropout
        if self.dropout_keep_prob is not None:
            cell_tmp = rnn.DropoutWrapper(cell_tmp, output_keep_prob=self.dropout_keep_prob)
        return cell_tmp

    def network(self):
        """
        RNN 网络搭建

        :return:
        """
        # 1. embedding layer
        with tf.name_scope('embedding'):
            if self.embedding_mat is None:
                self.Embedding = tf.Variable(tf.random_uniform([self.vocab_size, self.embedding_dims],
                                                       -1., 1.), name='Embedding')
                self.embedded_chars = tf.nn.embedding_lookup(self.Embedding, self.input_x)

        # 2. RNN hidden layer
        with tf.name_scope('rnn'):
            if self.num_layer > 1:
                cells = rnn.MultiRNNCell([self.witch_cell()] * self.num_layer, state_is_tuple=True)
            else:
                cells = self.witch_cell()
            # outputs:[batch, timestep_size, hidden_size]
            # state:[layer_num, 2, batch_size, hidden_size]
            outputs, _ = tf.nn.dynamic_rnn(cells, self.embedded_chars, dtype=tf.float32)

            # 取出最后一个状态的输出
            h_state = outputs[:, -1, :]
        # 3. FC and softmax layer
        with tf.name_scope('output'):
            # fc = tf.layers.dense(h_state, self.hidden_unit, name='FC')
            # fc = tf.contrib.layers.dropout(self.dropout_keep_prob)
            # fc = tf.nn.relu(fc)
            #
            # # softmax
            # self.logits = tf.layers.dense(fc, self.num_classes, name='softmax')
            # self.prediced = tf.argmax(tf.nn.softmax(self.logits), 1)
            self.W = tf.Variable(tf.truncated_normal([self.hidden_unit, self.num_classes], stddev=0.1), dtype=tf.float32, name='W')
            self.b = tf.Variable(tf.constant(0.1, shape=[self.num_classes]),dtype=tf.float32, name='b')

            self.logits = tf.nn.softmax(tf.matmul(h_state, self.W) + self.b)
            

        # loss
        with tf.name_scope('loss'):
            cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=self.logits, labels=self.input_y)
            self.loss = tf.reduce_mean(cross_entropy)
            # l2_loss = tf.add_n([tf.nn.l2_loss(v) for v in tf.trainable_variables()
            #                     if 'bias' not in v.name]) * self.l2_reg_lambda

            self.l2_loss += tf.nn.l2_loss(self.W)
            self.l2_loss += tf.nn.l2_loss(self.b)
            self.loss += self.l2_loss

        with tf.name_scope('accuracy'):
            predicted = tf.equal(tf.argmax(self.logits, 1), 
                                      tf.arg_max(self.input_y, 1))
            self.accuracy = tf.reduce_mean(tf.cast(predicted, dtype=tf.float32))

#### 4. 模型训练

In [ ]:
with tf.Graph().as_default():
    session_conf = tf.ConfigProto(
      allow_soft_placement=FLAGS.allow_soft_placement,
      log_device_placement=FLAGS.log_device_placement)
    sess = tf.Session(config=session_conf)
    with sess.as_default():
        # 构建rnn 节点
        rnns = RNNs(
            embedding_mat=None,
            embedding_dims=FLAGS.embedding_dim,
            vocab_size=len(word_index),
            non_static=False,
            hidden_unit=FLAGS.hidden_unit,
            sequence_length=x_train.shape[1],
            num_classes=y_train.shape[1],
            cell=FLAGS.cell,
            num_layers=FLAGS.num_layer,
            l2_reg_lambda=FLAGS.l2_reg_lambda)

        # Define Training procedure
        global_step = tf.Variable(0, name="global_step", trainable=False)
        # 优化算法
        optimizer = tf.train.AdamOptimizer(1e-3)
        grads_and_vars = optimizer.compute_gradients(rnns.loss)

        train_op = optimizer.apply_gradients(grads_and_vars, global_step=global_step)

        # Keep track of gradient values and sparsity (optional)
        grad_summaries = []
        for g, v in grads_and_vars:
            if g is not None:
                grad_hist_summary = tf.summary.histogram("{}/grad/hist".format(v.name), g)
                sparsity_summary = tf.summary.scalar("{}/grad/sparsity".format(v.name),
                                                     tf.nn.zero_fraction(g))
                grad_summaries.append(grad_hist_summary)
                grad_summaries.append(sparsity_summary)
        grad_summaries_merged = tf.summary.merge(grad_summaries)

        # Output directory for models and summaries
        timestamp = str(int(time.time()))
        out_dir = os.path.abspath(os.path.join(os.path.curdir, "runs", timestamp))
        print("Writing to {}\n".format(out_dir))

        # Summaries for loss and accuracy
        loss_summary = tf.summary.scalar("loss", rnns.loss)
        acc_summary = tf.summary.scalar("accuracy", rnns.accuracy)

        # Train Summaries
        train_summary_op = tf.summary.merge([loss_summary, acc_summary, grad_summaries_merged])
        train_summary_dir = os.path.join(out_dir, "summaries", "train")
        train_summary_writer = tf.summary.FileWriter(train_summary_dir, sess.graph)

        # Dev summaries
        dev_summary_op = tf.summary.merge([loss_summary, acc_summary])
        dev_summary_dir = os.path.join(out_dir, "summaries", "dev")
        dev_summary_writer = tf.summary.FileWriter(dev_summary_dir, sess.graph)

        # Checkpoint directory. Tensorflow assumes this directory already exists so we need to create it
        checkpoint_dir = os.path.abspath(os.path.join(out_dir, "checkpoints"))
        checkpoint_prefix = os.path.join(checkpoint_dir, "model")
        if not os.path.exists(checkpoint_dir):
            os.makedirs(checkpoint_dir)
        saver = tf.train.Saver(tf.global_variables(), max_to_keep=FLAGS.num_checkpoints)

        # Write vocabulary
        #vocab_processor.save(os.path.join(out_dir, "vocab"))

        # Initialize all variables
        sess.run(tf.global_variables_initializer())

        def train_step(x_batch, y_batch):
            """
            A single training step
            """
            feed_dict = {
              rnns.input_x: x_batch,
              rnns.input_y: y_batch,
              rnns.dropout_keep_prob: FLAGS.dropout_keep_prob
            }
            # 执行 节点操作
            _, step, summaries, loss, accuracy = sess.run(
                [train_op, global_step, train_summary_op, rnns.loss, rnns.accuracy],
                feed_dict)

            time_str = datetime.datetime.now().isoformat()
            if step % 20 == 0:
                print("{}: step {}, loss {:g}, acc {:g}".format(time_str, step, loss, accuracy))
            train_summary_writer.add_summary(summaries, step)

        def dev_step(x_batch, y_batch, writer=None):
            """
            Evaluates model on a dev set
            """
            feed_dict = {
              rnns.input_x: x_batch,
              rnns.input_y: y_batch,
              rnns.dropout_keep_prob: 1.0
            }
            step, summaries, loss, accuracy = sess.run(
                [global_step, dev_summary_op, rnns.loss, rnns.accuracy],
                feed_dict)
            time_str = datetime.datetime.now().isoformat()
            print("{}: step {}, loss {:g}, acc {:g}".format(time_str, step, loss, accuracy))
            if writer:
                writer.add_summary(summaries, step)
            return loss, accuracy
        # Generate batches
        batches = data_helper.batch_iter(
            list(zip(x_train, y_train)), FLAGS.batch_size, FLAGS.num_epochs)
        # Training loop. For each batch...
        best_acc = 0.0
        best_step = 0
        for batch in batches:
            x_batch, y_batch = zip(*batch)
            train_step(x_batch, y_batch)
            # 更新全局步数
            current_step = tf.train.global_step(sess, global_step)
            # 计算评估结果
            if current_step % FLAGS.evaluate_every == 0:
                print("\nEvaluation:")
                loss_, accuracy_ = dev_step(x_dev, y_dev, writer=dev_summary_writer)
                if accuracy_ > best_acc:
                    best_acc = accuracy_
                    best_step = current_step
                print("")
            # 保存模型计算结果
            if current_step % FLAGS.checkpoint_every == 0:
                path = saver.save(sess, checkpoint_prefix, global_step=current_step)
                print("Saved model checkpoint to {}\n".format(path))

        print('\nBset dev at {}, accuray {:g}'.format(best_step, best_acc))